# Charts of food combinations
This Jupyter Notebook aims at creating polar charts and food tables for the food combinations found (for now, found incrementally---ideally it should come from an algorithm that should be more objective, see the Optimization notebook)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import linalg, optimize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import dataframe_image as dfi

In [2]:
nut_calc = pd.read_excel('Nutrition Calculator.xlsx', sheet_name='Nutrition Calculator (p100kcal)',
                         skiprows=(0,1,3), index_col=0)
nut_needs = pd.read_excel('Nutrition Calculator.xlsx', sheet_name='Human Nutritional Needs',
                          nrows=5, skiprows=(0,2), index_col=0)

In [3]:
nut_calc = nut_calc.fillna(0)

In [4]:
nut_needs

,Protein,Fat total,Saturated fats,Trans fats,Carbohydrates,Fiber,Sugars,Vitamin A,Vitamin E,Vitamin D,...,Lysine,Methionine,Cysteine,Phenylalanine,Tyrosine,Threonine,Tryptophan,Valine,Omega-3 (ALA),Omega-6 (LA)
Lower (severe risk),46.00,35.00,1.000000e-99,1.000000e-99,5.000000e+01,2.500000e+01,1.000000e-99,5.000000e+02,1.000000e+04,3.8,...,1.860000e+00,6.200000e-01,2.480000e-01,9.300000e-01,2.480000e-01,9.300000e-01,2.480000e-01,1.612000e+00,1.350000e+00,1.000000e+01
Lower (moderate risk),51.00,46.67,1.000000e-99,1.000000e-99,1.300000e+02,2.500000e+01,1.000000e-99,7.000000e+02,1.500000e+04,15.0,...,1.860000e+00,6.200000e-01,2.480000e-01,9.300000e-01,2.480000e-01,9.300000e-01,2.480000e-01,1.612000e+00,2.000000e+00,1.450000e+01
Higher (safe),78.75,70.00,2.333000e+01,2.300000e+00,4.000000e+02,7.000000e+01,5.250000e+01,3.000000e+03,3.000000e+05,250.0,...,7.440000e+00,2.480000e+00,9.920000e-01,3.720000e+00,9.920000e-01,3.720000e+00,9.920000e-01,6.448000e+00,2.800000e+00,1.867000e+01
Higher (moderate risk),140.00,81.67,2.333000e+01,2.300000e+00,5.000000e+02,5.000000e+02,1.312500e+02,8.000000e+03,1.000000e+06,1925.0,...,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,4.670000e+00,2.333000e+01
Higher (severe risk),400.00,200.00,2.000000e+02,2.000000e+02,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,15000.0,...,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99


In [5]:
nut_calc

,Notes,Source,Energy,Protein digestibility,Protein,Fat total,Saturated fats,Trans fats,Carbohydrates,Fiber,...,Lysine,Methionine,Cysteine,Phenylalanine,Tyrosine,Threonine,Tryptophan,Valine,Omega-3 (ALA),Omega-6 (LA)
Food,,,,,,,,,,,,,,,,,,,,,
"Fish, tuna, fresh, skipjack, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,103.000000,0.900,21.359223,0.980583,0.318447,0.000000,0.000000,0.000000,...,1.961165,0.632039,0.229126,0.833981,0.721359,0.935922,0.238835,1.100000,0.000000,0.015534
"Cattle (meat, mechanically separated)","mechanically separated, raw",https://fdc.nal.usda.gov/fdc-app.html#/food-de...,276.000000,0.950,5.423913,8.521739,4.268116,0.000000,0.000000,0.000000,...,0.420652,0.154348,0.083333,0.233333,0.138043,0.171739,0.060870,0.331522,0.061594,0.213768
"Beef, variety meats and by-products, liver, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,135.000000,0.950,15.081481,2.688889,0.913333,0.125926,2.881481,0.000000,...,1.190370,0.402222,0.278519,0.802963,0.597778,0.643704,0.194815,0.933333,0.005185,0.221481
"Chicken, broilers or fryers, meat and skin and giblets and neck, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,213.000000,1.000,8.605634,6.962441,1.990610,0.000000,0.061033,0.000000,...,0.690141,0.226291,0.115493,0.337089,0.275117,0.356808,0.095305,0.419249,0.000000,1.328638
"Mushrooms, shiitake, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,34.000000,0.690,6.588235,1.441176,0.000000,0.000000,19.970588,7.352941,...,0.394118,0.097059,0.064706,0.326471,0.229412,0.394118,0.032353,0.426471,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Beef, tripe, cooked, simmered",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,94.000000,0.971,12.446809,4.308511,1.446809,0.212766,2.117021,0.000000,...,0.724404,0.296856,0.125090,0.413234,0.465511,0.470489,0.118245,0.000000,0.008511,0.145745
Cattle (organs),calculated from individual organ nutritional v...,0,139.808421,0.950,14.722854,3.494903,1.369257,0.144928,1.504420,0.000000,...,1.033903,0.363285,0.127949,0.387572,0.329675,0.606136,0.173915,0.180805,0.012212,0.175925
Lignocellulosic sugar,sugar from cellulose,0,400.000000,0.000,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
nutrients = [['Protein', 'Fat total', 'Saturated fats', 'Trans fats', 'Omega-3 (ALA)', 'Omega-6 (LA)',
              'Carbohydrates','Fiber', 'Sugars'],
             ['Vitamin A', 'Vitamin E', 'Vitamin D', 'Vitamin C', 'Thiamine (B1)', 'Riboflavin (B2)',
              'Niacin (B3)', 'Vitamin B6','Vitamin B12', 'Vitamin K', 'Folate (B9)', 'Pantothenic Acid (B5)'],
             ['Calcium', 'Iron', 'Magnesium', 'Phosphorus', 'Potassium', 'Sodium', 'Zinc', 'Copper',
              'Manganese', 'Selenium', 'Iodine'],
             ['Histidine', 'Isoleucine', 'Leucine', 'Lysine', 'Methionine', 'Cysteine', 
              'Phenylalanine', 'Tyrosine', 'Threonine', 'Tryptophan', 'Valine']]

diets_excel=pd.ExcelFile("Diets by period.xlsx")

for sheet in diets_excel.sheet_names:
    diets = pd.read_excel("Diets by period.xlsx", index_col=0, sheet_name=sheet)
    diets = diets.fillna(0)
    diets = diets.drop('Total:')
    food_subset_2 = diets.index
    nut_calc_subset=nut_calc.loc[list(food_subset_2)]
    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2)
    nut_calc_subset_t=nut_calc_subset[nut_needs.columns].T
    diet = nut_calc_subset_t @ (diets['kcal']/100)
    diet_df = pd.DataFrame(diet, columns=['value'])
    print(sheet, '\n-------\n', diet)
    
    SRAI = []
    MRAI = []
    AI = []
    danger = []
    for i in range(len(diet)): # for the solution, divide in categories for each nutrient
        nutrient=diet[i]
        nut=diet.index[i]
        lower_severe = nut_needs.at['Lower (severe risk)', nut]
        lower_moderate = nut_needs.at['Lower (moderate risk)', nut]
        higher_safe = nut_needs.at['Higher (safe)', nut]
        higher_moderate = nut_needs.at['Higher (moderate risk)', nut]
        higher_severe = nut_needs.at['Higher (severe risk)', nut]
        if (nutrient>= lower_moderate) and (nutrient<=higher_safe):
            AI += [nut]
        elif (nutrient==0):
            danger+= [nut + ' 0%']
        elif (nutrient>higher_severe):
            danger+= [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
        elif (nutrient<lower_severe):
            SRAI += [nut + ' ' + str(round(nutrient/lower_moderate*100))+'%']
        elif (nutrient>higher_moderate):
            SRAI += [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
        elif (nutrient< lower_moderate):
            MRAI += [nut + ' ' + str(round(nutrient/lower_moderate*100))+'%']
        elif (nutrient>higher_safe):
            MRAI += [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
    print('AI', AI, '\n\nMRAI', MRAI, '\n\nSRAI', SRAI, '\n\nDanger', danger)
    
    diet_df['lower adequate intake'] = pd.Series(nut_needs.T['Lower (moderate risk)'])
    diet_df['upper adequate intake'] = pd.Series(nut_needs.T['Higher (safe)'])
    diet_df['% lower adequate intake'] = diet_df['value']/diet_df['lower adequate intake']
    diet_df.loc[diet_df['% lower adequate intake']>1, '% lower adequate intake']=1
    diet_df['% upper adequate intake'] = diet_df['value']/diet_df['upper adequate intake']
    diet_macro = diet_df.loc[nutrients[0]]
    diet_macro = diet_macro.append(diet_macro.iloc[[0]])
    fig.add_trace(go.Scatterpolar(r=diet_macro['% lower adequate intake'], theta=diet_macro.index,
                                  name='% lower adequate intake', line_color='#00CC96'), 1, 1)
    fig.add_trace(go.Scatterpolar(r=diet_macro['% upper adequate intake'], theta=diet_macro.index,
                                  name='% upper adequate intake', line_color='#EF553B'), 1, 1)
    
    diet_vit = diet_df.loc[nutrients[1]]
    diet_vit = diet_vit.append(diet_vit.iloc[[0]])
    fig.add_trace(go.Scatterpolar(r=diet_vit['% lower adequate intake'], theta=diet_vit.index,
                                  name='% lower adequate intake', line_color='#00CC96'), 1, 2)
    fig.add_trace(go.Scatterpolar(r=diet_vit['% upper adequate intake'], theta=diet_vit.index,
                                  name='% upper adequate intake', line_color='#EF553B'), 1, 2)
    diet_min = diet_df.loc[nutrients[2]]
    diet_min = diet_min.append(diet_min.iloc[[0]])
    fig.add_trace(go.Scatterpolar(r=diet_min['% lower adequate intake'], theta=diet_min.index,
                                  name='% lower adequate intake', line_color='#00CC96'), 2, 1)
    fig.add_trace(go.Scatterpolar(r=diet_min['% upper adequate intake'], theta=diet_min.index,
                                  name='% upper adequate intake', line_color='#EF553B'), 2, 1)
    diet_aa = diet_df.loc[nutrients[3]]
    diet_aa = diet_aa.append(diet_aa.iloc[[0]])
    fig.add_trace(go.Scatterpolar(r=diet_aa['% lower adequate intake'], theta=diet_aa.index,
                                  name='% lower adequate intake', line_color='#00CC96'), 2, 2)
    fig.add_trace(go.Scatterpolar(r=diet_aa['% upper adequate intake'], theta=diet_aa.index,
                                  name='% upper adequate intake', line_color='#EF553B'), 2, 2)
    max_val = 2
    text = sheet.split('-')
    title='Period '+text[0]
    if text[1]=='BCS':
        title+=' - optimistic scenario'
    elif text[1][:2]=='wo':
        title+=' - without'+text[1][2:]
        
    fig.update_layout(
        height=800, width=1050, title_text=title,
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, max_val]
        ),
      angularaxis = dict(
        rotation = 90,
        direction = "clockwise"
      )),
      polar2=dict(
        radialaxis=dict(
          visible=True,
          range=[0, max_val]
        ),
          angularaxis = dict(
        rotation = 90,
        direction = "clockwise"
      )),
      polar3=dict(
        radialaxis=dict(
          visible=True,
          range=[0, max_val]
        ),
      angularaxis = dict(
        rotation = 90,
        direction = "clockwise"
      )),
      polar4=dict(
        radialaxis=dict(
          visible=True,
          range=[0, max_val]
        ),
      angularaxis = dict(
        rotation = 90,
        direction = "clockwise"
      )),
      showlegend=False
    )
    fig.show()
    fig.write_image(title+'.jpeg')

I-BCS 
-------
 Protein                    101.823661
Fat total                   84.600959
Saturated fats              17.396798
Trans fats                   0.228855
Carbohydrates              240.178705
Fiber                       28.635557
Sugars                      24.375946
Vitamin A                 1015.678639
Vitamin E                10226.114819
Vitamin D                    0.377435
Vitamin C                   62.792522
Thiamine (B1)                1.707000
Riboflavin (B2)              2.474202
Niacin (B3)                 39.997183
Vitamin B6                   2.594176
Vitamin B12                 10.728057
Vitamin K                   94.312996
Folate (B9)                388.354328
Pantothenic Acid (B5)        7.295744
Calcium                    907.923764
Iron                        18.922269
Magnesium                  629.412146
Phosphorus                1534.027100
Potassium                 5043.385233
Sodium                     536.292449
Zinc                        12.917

I-wo cereals 
-------
 Protein                   129.643678
Fat total                  74.731360
Saturated fats             35.047983
Trans fats                  0.362319
Carbohydrates             223.747765
Fiber                      13.448276
Sugars                     77.495572
Vitamin A                4591.903964
Vitamin E                1872.262373
Vitamin D                   1.067503
Vitamin C                 112.238470
Thiamine (B1)               1.356086
Riboflavin (B2)             3.648813
Niacin (B3)                53.579521
Vitamin B6                  3.739632
Vitamin B12                46.402143
Vitamin K                  19.463463
Folate (B9)               250.742753
Pantothenic Acid (B5)      11.313467
Calcium                  1723.257039
Iron                       28.207520
Magnesium                 343.537945
Phosphorus               2029.968204
Potassium                4929.753109
Sodium                    645.533369
Zinc                       18.647468
Copper         

II-BCS 
-------
 Protein                    121.298443
Fat total                   61.128901
Saturated fats              13.321143
Trans fats                   0.234294
Carbohydrates              274.501484
Fiber                       43.532230
Sugars                      20.844864
Vitamin A                 2463.957086
Vitamin E                11467.457307
Vitamin D                    2.222368
Vitamin C                  175.680903
Thiamine (B1)                1.941472
Riboflavin (B2)              4.112879
Niacin (B3)                 51.625479
Vitamin B6                   3.629676
Vitamin B12                 19.182565
Vitamin K                   83.366136
Folate (B9)                696.661953
Pantothenic Acid (B5)        9.829285
Calcium                   1035.492850
Iron                        59.550778
Magnesium                  967.327782
Phosphorus                1680.656737
Potassium                 7797.159693
Sodium                    3075.811314
Zinc                        15.18

II-wo seaweed 
-------
 Protein                   107.021100
Fat total                  60.808764
Saturated fats             14.253388
Trans fats                  0.234294
Carbohydrates             286.330868
Fiber                      24.451048
Sugars                     49.244773
Vitamin A                1894.407053
Vitamin E                6993.271245
Vitamin D                   0.508968
Vitamin C                 103.171250
Thiamine (B1)               1.647441
Riboflavin (B2)             2.593205
Niacin (B3)                48.664481
Vitamin B6                  3.347854
Vitamin B12                18.248329
Vitamin K                  70.410743
Folate (B9)               356.631941
Pantothenic Acid (B5)       9.491752
Calcium                   838.815843
Iron                       18.920279
Magnesium                 560.161166
Phosphorus               1560.072314
Potassium                5505.362601
Sodium                    533.294836
Zinc                       12.919172
Copper        

III-BCS 
-------
 Protein                     99.372681
Fat total                   62.446030
Saturated fats               7.328228
Trans fats                   0.134050
Carbohydrates              300.162304
Fiber                       49.970570
Sugars                      32.377505
Vitamin A                  616.970676
Vitamin E                13501.590952
Vitamin D                    1.496139
Vitamin C                  119.256001
Thiamine (B1)                2.547694
Riboflavin (B2)              5.400171
Niacin (B3)                 27.102394
Vitamin B6                   1.920850
Vitamin B12                  1.085907
Vitamin K                  130.851440
Folate (B9)                967.459089
Pantothenic Acid (B5)        7.249519
Calcium                    735.925934
Iron                        59.637580
Magnesium                 1169.375949
Phosphorus                1981.706306
Potassium                 6091.605795
Sodium                    3277.409894
Zinc                        12.9

III-wo greenhouses 
-------
 Protein                     73.894017
Fat total                   58.012766
Saturated fats               5.357008
Trans fats                   0.178733
Carbohydrates              321.813522
Fiber                       50.368985
Sugars                      49.815517
Vitamin A                  581.313919
Vitamin E                13805.841257
Vitamin D                    1.555985
Vitamin C                  161.289784
Thiamine (B1)                2.380192
Riboflavin (B2)              4.881039
Niacin (B3)                 26.860577
Vitamin B6                   2.844890
Vitamin B12                  1.129344
Vitamin K                   81.373315
Folate (B9)                540.877865
Pantothenic Acid (B5)        5.344268
Calcium                    522.343553
Iron                        57.686012
Magnesium                  849.239567
Phosphorus                1655.025268
Potassium                 5552.272090
Sodium                    3165.998031
Zinc                 

In [7]:
diets_excel=pd.ExcelFile("Diets by period.xlsx")

for sheet in diets_excel.sheet_names:
    diets = pd.read_excel("Diets by period.xlsx", index_col=0, sheet_name=sheet)
    diets = diets.fillna(0)
    
    text = sheet.split('-')
    title='Period '+text[0]
    if text[1]=='BCS':
        title+=' - optimistic scenario'
    elif text[1][:2]=='wo':
        title+=' - without'+text[1][2:]

    dfi.export(diets, title+' table.png')